In [ ]:
import glob
import numpy as np
from astropy.io import fits

 Due to the observatory regulations, the observations with KMOS are needed to be splitted
 into two halves. KMOS data is divided into two halves in a somewhat complex manner, requiring their 
 merging to obtain a single set of 3-dimensional data (i.e., a ‘master cube’) before further analysis. 
 Each half is sliced, alternating between data and empty strips. Additionally, each half has its own 
 transformation matrix between pixel position and celestial coordinates, making the merging process non-trivial.

In [ ]:
path = 'C:\\Users\\pablo\\Desktop\\TFM\\data\\first_kmos\\'   # path were the two halves are

fileList = sorted(glob.glob(path+'*.fits'))    # List of the two .fits files sorted
im_redu = []                                   # List where to save the [1] component of every hdu
for i in range(len(fileList)):
    hdu = fits.open(fileList[i])               # From each FITS we work on the 2nd element, which is the SCI
    data = hdu[1].data 
    im_redu.append(data) 

wcs_half1 = WCS(fits.open(fileList[0])[1].header)  # Each half has a different WCS
wcs_half2 = WCS(fits.open(fileList[1])[1].header)

Each slice has 27 rows, that's the origin of the "27" that will appear in the following cell.
Here the idea is the next one: master_ima is an image corresponding to the first half + the extra slice
from the second half, hence "correcting" the offset between these two halves. 
Then run over all its pixels, if we find nan, then replace it with the value
of the same position in the second half. 

In [ ]:
# This function merges two halves of kmos images into one
# The images must have 3 axis. The syntax is: [z, y, x]
def make_master_cube(half1, half2):

    master_cube = []
    for k in range(len(half1[:, 0, 0])):
        master_ima = np.concatenate((half2[k, 0:27, :], half1[k, :, :] ))   # This image has the dimension of the full image and contains 
                                                                                 # the first half and the first strip of the second half
                                                                                # (that actually is the last due to the origin=lower in plots)
        for j in range(len(half1[0, :, 0])):       # Run Y pixels  
            for i in range(len(half1[0, 0, :])):   # Run X pixels
                if np.isnan(master_ima[j,i]):      # If a pixel is nan in the master_image image, replace its value for the one in the 2nd half     
                    master_ima[j,i] = half2[k,j,i]
                    
        master_cube.append(master_ima)             # master_ima is for a single channel, master_cube contains all channels (all wavelength)

    return master_cube

In [ ]:
# This procedure usually last (for the data of this work) about an hour
master_cube = make_master_cube(im_redu[0], im_redu[1])